# IEEEXTREME-8

## Kabloom - (100/100) <br /><br />
link: https://www.hackerrank.com/contests/ieeextreme-challenges/challenges/kabloom <br /><br />
Time: O(m * n) <br />
Space: O(m * n) <br />
The solution is using LCS with a small adjustment since each char has a differenet value (instead of all being 1)

In [ ]:
points = {'A': 20, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'J': 15, 'Q': 15, 'K': 15,
          'R': 50}


def calcPoints(a, b):
    return points[b] if a == "R" and b != "R" else points[a]


def lcs(A, B, n):
    mat = [[0] * (n + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            mat[i][j] = max(mat[i - 1][j], mat[i][j - 1])
            if A[i] == B[j] or A[i] == "R" or B[j] == "R":
                mat[i][j] = max(mat[i][j], mat[i - 1][j - 1] + calcPoints(A[i], B[j]))
                
    return mat[-1][-1] * 2


def main():
    n = int(input())
    while n != 0:
        A = ["None"] + [x for x in input().split()]
        B = ["None"] + [x for x in input().split()]
        print(lcs(A, B, n))
        n = int(input())


if __name__ == "__main__":
    main()

# IEEEXTREME-9

## Pattern 3 - (100/100)<br /> <br />

link:  https://www.hackerrank.com/contests/ieeextreme-challenges/challenges/pattern-3<br /><br />
Time: O(n) <br />
Space: O(n) <br />
Using sliding window to find the repeating pattern inside the string.

In [ ]:
# Sliding Window
def pattern(s):
    j, n = 0, len(s)
    p = [0] * n
    
    # i = fast, j = slow
    for i in range(1, n):
        
        # move j back to the last location without a pattern
        while j > 0 and s[j] != s[i]:
            j = p[j - 1]
            
        # slide the window to the right
        if s[j] == s[i]:
            j += 1
        
        # keep track of pattern
        p[i] = j
        
    return i - j + 1


def main():
    T = int(input())
    for i in range(T):
        print(pattern(input()))


if __name__ == "__main__":
    main()

# IEEEXTREME-10

## Goldbach's Second Conjecture - (100/100)<br />
link: https://www.hackerrank.com/contests/ieeextreme-challenges/challenges/goldbachs-second-conjecture/problem <br />
Credits: <br />
Sieve of Eratosthenes - rookieslab <br />
miller-rabin          - https://stackoverflow.com/questions/36522167/checking-primality-of-very-large-numbers-in-python
<br /><br />
The main idea is to find the largest prime number below the given odd n, <br />
then we can subtruct p from n which will give us an even number.<br />
On the even number we can apply the original Goldbach's Conjecture rule, and look for <br /> 2 prime numbers that add up to (n - p) in that range. <br />
For that I made a set of all primes in that range using the 'Sieve of Eratosthenes' algorithm, <br /> and from here it's very easy to look for two numbers that sum up to it in the set.

In [ ]:
from random import randint


def twoSum(primes, n):
    for p in primes:
        if n - p in primes:
            return p, n - p
    return None


# Sieve of Eratosthenes
def eratosthenes(n):
    is_prime = [0, 0] + [1]*(n - 2)
    i = 2
    while i*i <= n:
        if is_prime[i] == 0:
            i += 1
        else:
            j = 2*i
            while j < n:
                is_prime[j] = 0
                j += i
            i += 1
    return {i for i, x in enumerate(is_prime) if x}


# miller-rabin
def isPrime(n, k=5):
    if n < 2: return False
    for p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]:
        if n % p == 0: return n == p
    s, d = 0, n-1
    while d % 2 == 0:
        s, d = s+1, d//2
    for i in range(k):
        x = pow(randint(2, n-1), d, n)
        if x == 1 or x == n-1: continue
        for r in range(1, s):
            x = (x * x) % n
            if x == 1: return False
            if x == n-1: break
        else: return False
    return True


def biggestPrime(n):
    n = n-1 if n % 2 == 0 else n
    for i in range(n-4, 2, -2):
        if isPrime(i):
            return i
    return 2


def main():
    n = int(input())
    a = biggestPrime(n)
    primes = eratosthenes(n-a)
    b, c = twoSum(primes, n-a)
    print(f"{a} {b} {c}")


if __name__ == "__main__":
    main()

## Mysterious Maze <br />
link: https://www.hackerrank.com/contests/ieeextreme-challenges/challenges/mysterious-maze <br /><br />
Since backtracking is too slow, the main idea here is to build the path as it goes. <br />
When a door open on the top, it's value will be 2, on the bottom 3, and anywhere else 1. <br />
every 2 or 3 input will check its surroundings and "infect" the 1s next to it which will recursivly <br /> 
infect the 1s next to that and so on. every iteration we will check if the input has both 2 and 3 in its <br />
surroundings, and if so then it means the paths from the top and bottom have connected and we have a full path.

In [ ]:
# Unblock python's 1000 limit on recursion
import sys
sys.setrecursionlimit(10000)


# check if top and bottom are connected by a path
def checkConnection(maze, n, x, y):
    two = True if maze[x][y] == 2 else False
    three = True if maze[x][y] == 3 else False

    locations = ()
    if x != 0: locations += ((x-1, y),)
    if x != n-1: locations += ((x + 1, y),)
    if y != 0: locations += ((x, y - 1),)
    if y != n-1: locations += ((x, y + 1),)

    for i, j in locations:
        if two and three:
            break
        two = True if two or maze[i][j] == 2 else False
        three = True if three or maze[i][j] == 3 else False
    return two and three


# Update neighbors with known path
def updateNeighbors(maze, n, x, y, ID):
    if x > 0 and maze[x - 1][y] == 1:
        maze[x - 1][y] = ID
        updateNeighbors(maze, n, x-1, y, ID)
    if x < n - 1 and maze[x + 1][y] == 1:
        maze[x + 1][y] = ID
        updateNeighbors(maze, n, x+1, y, ID)
    if y > 0 and maze[x][y - 1] == 1:
        maze[x][y - 1] = ID
        updateNeighbors(maze, n, x, y-1, ID)
    if y < n - 1 and maze[x][y + 1] == 1:
        maze[x][y + 1] = ID
        updateNeighbors(maze, n, x, y+1, ID)


# Check if any neighbors are in a known path and update self
def updatedByNeighbors(maze, n, x, y):
    update = False
    if x > 0 and maze[x - 1][y] > 1:
        update = True
        maze[x][y] = maze[x - 1][y]
    elif x < n - 1 and maze[x + 1][y] > 1:
        update = True
        maze[x][y] = maze[x + 1][y]
    elif y > 0 and maze[x][y - 1] > 1:
        update = True
        maze[x][y] = maze[x][y - 1]
    elif y < n - 1 and maze[x][y + 1] > 1:
        update = True
        maze[x][y] = maze[x][y + 1]

    if update:
        updateNeighbors(maze, n, x, y, maze[x][y])


def solve(maze, n, x, y):
    if x == 0: maze[x][y] = 2
    elif x == n-1: maze[x][y] = 3
    else: maze[x][y] = 1

    if checkConnection(maze, n, x, y):
        return True

    if x == 0 or x == n-1:
        updateNeighbors(maze, n, x, y, maze[x][y])
    else:
        updatedByNeighbors(maze, n, x, y)


def main():
    n = int(input())
    maze = [[0] * n for _ in range(n)]
    i = 0
    while True:
        i += 1
        door = input()
        if door == "-1":
            print("-1")
            break
        else:
            x, y = map(int, door.split())
            x, y = x - 1, y - 1
            if solve(maze, n, x, y):
                print(i)
                break
            else:
                continue


if __name__ == "__main__":
    main()

# IEEEXTREME-11

## Rumour - (100/100)

link: https://csacademy.com/ieeextreme-practice/task/9ca8fafd184f553a903734761546a224/ <br />
Time:  max(O(logn), O(logm)) <br />
Space: O(1)

In [ ]:
def Rumour(a, b):
    # i, j: iterations to common ancestor
    i = j = 0
    while a != b:
        if a > b:
            a //= 2
            i += 1
        else:
            b //= 2
            j += 1
    
    return i + j


q = int(input())
for i in range(q):
    a, b = map(int, input().split())
    print(Rumour(a, b))

# IEEEXTREME-12

# IEEEXTREME-13

## Minimum Permutation - (100/100)
link: https://csacademy.com/ieeextreme-practice/task/minimum-permutation/ <br />
Time:  O(n + m) <br />
Space: O(n + m) <br /> <br />
Print the minimum number from set or list and continue on the same array, until minimum <br />
is from the other array, then do step 1 again

In [ ]:
def minPerm(N, M, l, s):
    res, i, j = [], 0, 0
    while i < N and j < M:
        if l[i] < s[j]:
            res.append(l[i])
            i += 1
        else:
            res.append(s[j])
            j += 1
    res += s[j:] if i == N else l[i:]
    return res

def main():
    N, M = map(int, input().split())
    l = map(int, input().split())
    s = sorted([int(x) for x in input().split()])
    print(*minPerm(N, M, l, s))
    
if __name__ == "__main__":
    main()

## Concentration Game - (100/100)
link: https://csacademy.com/ieeextreme-practice/task/concentration-game/ <br />
Time:  O(N) <br />
Space: O(N)

In [ ]:
# Flush the cards by and return their value
def flush(x, y):
    print(x, y, flush=True)
    return input()


# Remove the known card from deck and flush the match
def foundMatch(faceDownCards, x, y):
    faceDownCards.remove(y)
    flush(x, y)


# Add the card to known cards and move it to the back of the deck
def foundNewCard(foundCards, faceDownCards, a, x):
    faceDownCards.append(x)
    foundCards[a] = x


# Check if given card was seen already or its the first time
def checkCard(faceDownCards, foundCards, cardValue, cardIdx):
    # Found a card that we know of its twin
    if cardValue in foundCards:
        foundMatch(faceDownCards, cardIdx, foundCards[cardValue])
    # Found a card that we haven't found it's twin yet
    else:
        foundNewCard(foundCards, faceDownCards, cardValue, cardIdx)


def game(foundCards, faceDownCards):
    while faceDownCards:
        x = faceDownCards.pop(0)
        y = faceDownCards.pop(0)
        answer = flush(x, y)

        if answer != "MATCH":
            a, b = map(int, answer.split())
            checkCard(faceDownCards, foundCards, a, x)
            checkCard(faceDownCards, foundCards, b, y)
        
    print('-1')


def main():
    N = int(input())
    foundCards, faceDownCards = {}, [x + 1 for x in range(2 * N)]
    game(foundCards, faceDownCards)


if __name__ == "__main__":
    main()

# IEEEXTREME-14

## Hotel Wiring - (100/100)

link: https://csacademy.com/ieeextreme-practice/task/hotel-wiring/ <br />
Time:  O(nlogn) <br />
Space: O(n) <br /> <br />
sort the floors by floors with most disfunctional rooms first, <br />
then return the sum of functional rooms and (amount of rooms minus amount of disfunctional rooms) in first k floors

In [ ]:
def hotelWiring(rooms, N, K):
    return sum([x if i >= K else N - x for i, x in enumerate(sorted(rooms))])


def main():
    T = int(input())
    for i in range(T):
        rooms = []
        M, N, K = map(int, input().split())
        for i in range(M):
            rooms.append(int(input()))
        print(hotelWiring(rooms, N, K))
    
    
if __name__ == "__main__":
    main()

## Game of Life 2020 (100/100)
link: https://csacademy.com/ieeextreme-practice/task/conway-gameoflife/ <br />
Time:  O(M * (N^2)) <br />
Space: O(N^2)

In [ ]:
from copy import deepcopy


def printResult(result):
    for i in range(len(result)):
        print(''.join(str(x) for x in result[i]))


# Returns how many live neighbors there are
# There are 4 possible neighbor slots: above, below, right, left. Below the x's are neighbors to the 1
# 0 x 0
# x 1 x
# 0 x 0
def countNeighbors(cells, N, i, j):
    return cells[(i + 1) % N][j] + cells[i][(j + 1) % N] + cells[i - 1][j] + cells[i][j - 1]


# do M (M = generations) iterations on the matrix, changing each cell according to the rules.
def gameOfLife(mat, N, M, deadRules, liveRules):
    cells = deepcopy(mat)

    for m in range(M):
        for i in range(N):
            for j in range(N):
                neighbors = countNeighbors(mat, N, i, j)
                if mat[i][j] == 1 and neighbors not in liveRules:
                    cells[i][j] = 0
                elif mat[i][j] == 0 and neighbors in deadRules:
                    cells[i][j] = 1
        mat = deepcopy(cells)
    return mat


def main():
    deadRules, liveRules = map(str, input().split(";"))
    deadRules = {i for i, x in enumerate(list(deadRules)) if x == "1"}
    liveRules = {i for i, x in enumerate(list(liveRules)) if x == "1"}
    N, M = map(int, input().split())
    mat = [list(map(int, input())) for i in range(N)]
    printResult(gameOfLife(mat, N, M, deadRules, liveRules))


if __name__ == "__main__":
    main()

## Crafting Wooden Tables - (100/100)

Time:  O(1) <br />
Space: O(1)

In [ ]:
# C = Pieces of woods required to craft a table
# N = Pocket slots (each slot can hold a single table or P pieces of wood)
# P = Amount of pieces of wood that can fit into a pocket
# W = Total of pieces of wood availble

def calc(C, N, P, W):
    
    # if the amount of wood needed for one table is greater than the pocket size,
    # then we will take the minimum of amount of pockets, and amount of tables we can make
    if C >= P:
        return min(N, W//C)
        
    # if the amount of wood needed for one table is lesser than the pocket size,
    # then we also consider the slot 
    else:
        return min(N, W//C, (N*P-W) // (P-C))

C, N, P, W = map(int, input().split())
print(calc(C, N, P, W))

# IEEEXTREME-15

## Doctor's Appointments <br />
https://csacademy.com/ieeextreme-practice/task/appointment <br /><br />
First we sort the appointments by start date (reverse so we can use pop which is O(1) instead of pop(0) which is O(n)).
Then we push them into a min heap which is sorted by end date (closest on top), <br />
and then start pluging them in inside the result list. <br />
If the heap is empty at any iteration, it means that no one can come today, so result is immposible. <br />
if 'i' is bigger than the top of the heap, the end date is late so not everyone will make it.

In [ ]:
from heapq import heappush, heappop


def printResult(result):
    print(*result) if result else print("impossible")


def solve(s, n):
    # sort by start day
    s.sort(reverse=True)
    heap, result = [], []

    for i in range(1, n+1):

        # remove earlier appointments that start date in this day
        # then push to min heap by end date
        while len(s) > 0 and s[-1][0] <= i:
            _, endDate, idx = s.pop()
            heappush(heap, [endDate, idx])
        
        # no one to put in today's appointment, so the rest won't fit
        if not heap:
            result = []
            break
        else:
            # take out next appointment and its last date
            endDate, idx = heappop(heap)

            # no time for this appointment, therefore impossible
            if i > endDate:
                result = []
                break

            # next appointment in line
            result.append(idx)

    return result


def main():
    T = int(input())
    for i in range(T):
        N = int(input())
        schedule = []
        for j in range(1, N+1):
            schedule.append([int(x) for x in input().split()] + [j])
        printResult(solve(schedule, N))


if __name__ == "__main__":
    main()